In [1]:
import csv
import re
import nltk
import Sastrawi
import sklearn
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np

In [2]:
df = pd.read_csv("dataset\labeled_data.csv")

In [3]:
# ------ Case Folding --------
# gunakan fungsi Series.str.lower() pada Pandas
df['tweets'] = df['tweets'].str.lower()

print('Case Folding Result : \n')
print(df['tweets'].head(10))
print('\n\n\n')

tweets  label  \
0     Polres Tanjung Balai Laksanakan Vaksinasi http...    1.0   
1     Dukung Program Presiden Jokowi 1 Juta Vaksinas...    1.0   
2     Panglima TNI Marsekal TNI Hadi Tjahjanto menye...    1.0   
3     Polda Lampung menggelar vaksinasi covid-19 unt...    1.0   
4     Bupati Kobar Apresiasi Perusahaan Laksanakan V...    1.0   
...                                                 ...    ...   
998   Grab Luncurkan Pusat Vaksinasi Ramah Penyandan...    1.0   
999   Cegah Penyebaran Covid-19, Sentra Vaksinasi Ko...    1.0   
1000  Seneng liat animo teman-teman makin banyak yan...    1.0   
1001  Lonjakan Kasus Covid-19 Terjadi, Kemenkes Hany...    0.0   
1002  @mrs_enci @thedufresne Asa pernah dibahas awal...    1.0   

                                           case_folding  
0     polres tanjung balai laksanakan vaksinasi http...  
1     dukung program presiden jokowi 1 juta vaksinas...  
2     panglima tni marsekal tni hadi tjahjanto menye...  
3     polda lampung menggelar vaksinasi covid-19 unt...  
4     bupati kobar apresiasi perusahaan laksanakan v...  
...                                                 ...  
998   grab luncurkan pusat vaksinasi ramah penyandan...  
999   cegah penyebaran covid-19, sentra vaksinasi ko...  
1000  seneng liat animo teman-teman makin banyak yan...  
1001  lonjakan kasus covid-19 terjadi, kemenkes hany...  
1002  @mrs_enci @thedufresne asa pernah dibahas awal...  

[1003 rows x 3 columns]

In [4]:
import string 

# import word_tokenize & FreqDist from NLTK
from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist

# ------ Tokenizing ---------

def remove_tweet_special(text):
    # Menghapus additional code
    text = text.replace('\\&gt\\&amp', '')
    
    # Menghapus www.* & https?://* 
    text = re.sub(r"https?\://\S+","", text)
    
    # Menghapus mention @
    text = re.sub(r"[@][\w_-]+","", text)
    
    # Menghapus hashtag #
    text = re.sub(r"[#][\w_-]+","", text)
    
    # Menghapus tanda baca
    text = re.sub(r"[^\w\s]"," ", text)
    
    # Menghapus angka
    text = re.sub(r"\d+"," ", text)
    
    # Menghapus spasi ganda
    text = re.sub(r" +"," ", text)
    
    # Menghapus spasi awal
    text = re.sub(r"^\s+","", text)
    
    # Menghapus spasi akhir
    text = re.sub(r"\s+$","",text)

    return text
                
df['tweets'] = df['tweets'].apply(remove_tweet_special)

# remove single char
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

df['tweets'] = df['tweets'].apply(remove_singl_char)

# NLTK word rokenize 
def word_tokenize_wrapper(text):
    return word_tokenize(text)

df['tweet_tokens'] = df['tweets'].apply(word_tokenize_wrapper)

print('Tokenizing Result : \n') 
print(df['tweet_tokens'].head(10))
print('\n\n\n')

Tokenizing Result : 

0      [polres, tanjung, balai, laksanakan, vaksinasi]
1    [dukung, program, presiden, jokowi, juta, vaks...
2    [panglima, tni, marsekal, tni, hadi, tjahjanto...
3    [polda, lampung, menggelar, vaksinasi, covid, ...
4    [bupati, kobar, apresiasi, perusahaan, laksana...
5    [rs, columbia, asia, indonesia, mendukung, pro...
6    [pengaman, pelaksanaan, vaksinasi, bagi, warga...
7    [kabupaten, di, kalbar, disebut, tak, serius, ...
8    [salah, banget, vaksinasi, sebelum, ujian, men...
9    [silahkan, simak, jadwal, vaksinasi, per, kelu...
Name: tweet_tokens, dtype: object






In [9]:
df

tweets  label  \
0             polres tanjung balai laksanakan vaksinasi    1.0   
1     dukung program presiden jokowi juta vaksinasi ...    1.0   
2     panglima tni marsekal tni hadi tjahjanto menye...    1.0   
3     polda lampung menggelar vaksinasi covid untuk ...    1.0   
4     bupati kobar apresiasi perusahaan laksanakan v...    1.0   
...                                                 ...    ...   
998   grab luncurkan pusat vaksinasi ramah penyandan...    1.0   
999   cegah penyebaran covid sentra vaksinasi kota s...    1.0   
1000  seneng liat animo teman teman makin banyak yan...    1.0   
1001  lonjakan kasus covid terjadi kemenkes hanya iz...    0.0   
1002  asa pernah dibahas awal ya militer diperbantuk...    1.0   

                                           tweet_tokens  
0       [polres, tanjung, balai, laksanakan, vaksinasi]  
1     [dukung, program, presiden, jokowi, juta, vaks...  
2     [panglima, tni, marsekal, tni, hadi, tjahjanto...  
3     [polda, lampung, menggelar, vaksinasi, covid, ...  
4     [bupati, kobar, apresiasi, perusahaan, laksana...  
...                                                 ...  
998   [grab, luncurkan, pusat, vaksinasi, ramah, pen...  
999   [cegah, penyebaran, covid, sentra, vaksinasi, ...  
1000  [seneng, liat, animo, teman, teman, makin, ban...  
1001  [lonjakan, kasus, covid, terjadi, kemenkes, ha...  
1002  [asa, pernah, dibahas, awal, ya, militer, dipe...  

[1003 rows x 3 columns]

In [5]:
df = df.dropna(axis=0, subset=['label'])
df

tweets  label  \
0             polres tanjung balai laksanakan vaksinasi    1.0   
1     dukung program presiden jokowi juta vaksinasi ...    1.0   
2     panglima tni marsekal tni hadi tjahjanto menye...    1.0   
3     polda lampung menggelar vaksinasi covid untuk ...    1.0   
4     bupati kobar apresiasi perusahaan laksanakan v...    1.0   
...                                                 ...    ...   
998   grab luncurkan pusat vaksinasi ramah penyandan...    1.0   
999   cegah penyebaran covid sentra vaksinasi kota s...    1.0   
1000  seneng liat animo teman teman makin banyak yan...    1.0   
1001  lonjakan kasus covid terjadi kemenkes hanya iz...    0.0   
1002  asa pernah dibahas awal ya militer diperbantuk...    1.0   

                                           tweet_tokens  
0       [polres, tanjung, balai, laksanakan, vaksinasi]  
1     [dukung, program, presiden, jokowi, juta, vaks...  
2     [panglima, tni, marsekal, tni, hadi, tjahjanto...  
3     [polda, lampung, menggelar, vaksinasi, covid, ...  
4     [bupati, kobar, apresiasi, perusahaan, laksana...  
...                                                 ...  
998   [grab, luncurkan, pusat, vaksinasi, ramah, pen...  
999   [cegah, penyebaran, covid, sentra, vaksinasi, ...  
1000  [seneng, liat, animo, teman, teman, makin, ban...  
1001  [lonjakan, kasus, covid, terjadi, kemenkes, ha...  
1002  [asa, pernah, dibahas, awal, ya, militer, dipe...  

[956 rows x 3 columns]

In [6]:
df.drop_duplicates(subset=['tweets'], inplace=True)
df

tweets  label  \
0             polres tanjung balai laksanakan vaksinasi    1.0   
1     dukung program presiden jokowi juta vaksinasi ...    1.0   
2     panglima tni marsekal tni hadi tjahjanto menye...    1.0   
3     polda lampung menggelar vaksinasi covid untuk ...    1.0   
4     bupati kobar apresiasi perusahaan laksanakan v...    1.0   
...                                                 ...    ...   
998   grab luncurkan pusat vaksinasi ramah penyandan...    1.0   
999   cegah penyebaran covid sentra vaksinasi kota s...    1.0   
1000  seneng liat animo teman teman makin banyak yan...    1.0   
1001  lonjakan kasus covid terjadi kemenkes hanya iz...    0.0   
1002  asa pernah dibahas awal ya militer diperbantuk...    1.0   

                                           tweet_tokens  
0       [polres, tanjung, balai, laksanakan, vaksinasi]  
1     [dukung, program, presiden, jokowi, juta, vaks...  
2     [panglima, tni, marsekal, tni, hadi, tjahjanto...  
3     [polda, lampung, menggelar, vaksinasi, covid, ...  
4     [bupati, kobar, apresiasi, perusahaan, laksana...  
...                                                 ...  
998   [grab, luncurkan, pusat, vaksinasi, ramah, pen...  
999   [cegah, penyebaran, covid, sentra, vaksinasi, ...  
1000  [seneng, liat, animo, teman, teman, makin, ban...  
1001  [lonjakan, kasus, covid, terjadi, kemenkes, ha...  
1002  [asa, pernah, dibahas, awal, ya, militer, dipe...  

[792 rows x 3 columns]

In [10]:
from nltk.corpus import stopwords

# ----------------------- get stopword from NLTK stopword -------------------------------
# get stopword indonesia
list_stopwords = stopwords.words('indonesian')


# ---------------------------- manualy add stopword  ------------------------------------
# append additional stopword
list_stopwords.extend(['ada', 'adalah', 'adlah', 'adlh', 'agak', 'agar', 'agung', 'aja', 'akan', 'anda', 'anu', 'apakah', 'apa', 
                       'april', 'antara', 'atau','amat', 'apalagi', 'angin', 'api', 'akbp', 'azis', 'andriansyah', 's', 'h', 
                       'i', 'k', 'm', 'babhinkamtibmas', 'babinsa', 'bagaimanapun', 'bagi', 'bahwa', 'bandara', 'bandung', 
                       'beberapa', 'bbrp', 'bisa', 'bs', 'begitu', 'boleh', 'bulan', 'bhabinkamtibmas', 'bhabinkantibmas', 
                       'bhayangkara', 'bhayangkari', 'bripka', 'dan', 'dia', 'di', 'dari', 'dalam', 'dlm', 'dengan', 'dgn', 
                       'daripada', 'drpd', 'demi', 'dimana', 'dmn', 'dapat', 'dpt', 'dsb', 'dst', 'dll', 'dahulu', 'dua', 
                       'dulunya', 'demikian', 'desa', 'ds', 'drs', 'digunakan',  'es', 'f', 'februari', 'faceshield', 'gbla', 
                       'gelora', 'gorontalo', 'gue', 'gw', 'gubernur', 'guna', 'hal', 'halo', 'haloo', 'halooo', 'hanya', 
                       'hari', 'harus', 'hujan', 'hi', 'hadi', 'tjahjanto', 'hum', 'ia', 'ini', 'itu', 'itulah', 'ingin', 
                       'jika', 'juga', 'jg', 'jumat', 'jokowi', 'januari', 'juni', 'juli', 'joko', 'kah', 'kabupaten', 'kab', 
                       'kalo', 'klo', 'kapolda', 'kapolri', 'kapolsek', 'kami', 'kamil', 'kamis', 'karena', 'krn', 'kbp', 'ke', 
                       'kecamatan', 'kec', 'kecuali', 'kelurahan', 'kel', 'kemarin', 'kmrn', 'kemenkes', 'kembali', 'kepada', 
                       'kpd', 'kereta', 'ketika', 'kita', 'kenapa', 'knp', 'kemana', 'kan', 'kok', 'kudus', 'lagi', 'lg', 
                       'lain', 'lautan', 'listyo', 'leksono', 'maka', 'maret', 'mari', 'marsekal', 'masih', 'masker', 'menurut', 
                       'msh', 'mereka', 'mrk', 'mengapa', 'melainkan', 'merupakan', 'mei', 'minggu', 'mobilitas', 'namun', 
                       'nanti', 'nggak', 'noh', 'oh', 'oleh', 'ok', 'orang', 'pada', 'pd', 'para', 'pakansari', 'pasti', 
                       'panglima', 'pemkab', 'pemerintah', 'polda', 'polisi', 'polres', 'polri', 'polsek', 'puskesmas', 'pula', 
                       'pmk', 'prabowo', 'presiden', 'rabu', 'rs', 'ridwan', 'saat', 'sabtu', 'sambil', 'sampai', 'satgas', 
                       'saja', 'saya', 'sy', 'sebab', 'sebagai', 'sebelum', 'sblm', 'sebetulnya', 'secara', 'sedangkan', 
                       'seharusnya', 'sehingga', 'sekitar', 'selagi', 'sementara', 'selain', 'seolah', 'seperti', 'seraya', 
                       'serta', 'sesuatu', 'sesudah', 'setelah', 'tadi', 'tapi', 'tp', 'tanpa', 'telah', 'tentang', 'ttg', 
                       'tentu', 'terhadap', 'terminal', 'tetapi', 'tidak', 'toh', 'tolong', 'tni', 'uea', 'untuk', 'utk', 
                       'wakapol', 'wakapolda', 'walau', 'widodo', 'ya', 'yaitu', 'yakni', 'yang', 'yg'])

# ----------------------- add stopword from txt file ------------------------------------
# read txt stopword using pandas
txt_stopword = pd.read_csv("datasetbismillah\stopwords.txt", names= ["stopwords"], header = None)

# convert stopword string to list & append additional stopword
list_stopwords.extend(txt_stopword["stopwords"][0].split(' '))

# ---------------------------------------------------------------------------------------

# convert list to dictionary
list_stopwords = set(list_stopwords)


# remove stopword pada list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

df['tweet_tokens_WSW'] = df['tweet_tokens'].apply(stopwords_removal) 

print('Stopwords Result : \n') 
print(df['tweet_tokens_WSW'].head(10))
print('\n\n\n')

Stopwords Result : 

0              [tanjung, balai, laksanakan, vaksinasi]
1    [dukung, program, juta, vaksinasi, lumajang, g...
2              [menyebut, lonjakan, covid, terkendali]
3    [lampung, menggelar, vaksinasi, covid, anggota...
4    [bupati, kobar, apresiasi, perusahaan, laksana...
5    [columbia, asia, indonesia, mendukung, program...
6    [pengaman, pelaksanaan, vaksinasi, warga, usia...
7                  [kalbar, serius, vaksinasi, lansia]
8        [salah, banget, vaksinasi, ujian, menangisss]
9    [silahkan, simak, jadwal, vaksinasi, ketinggal...
Name: tweet_tokens_WSW, dtype: object






In [14]:
df

tweets  label  \
0             polres tanjung balai laksanakan vaksinasi    1.0   
1     dukung program presiden jokowi juta vaksinasi ...    1.0   
2     panglima tni marsekal tni hadi tjahjanto menye...    1.0   
3     polda lampung menggelar vaksinasi covid untuk ...    1.0   
4     bupati kobar apresiasi perusahaan laksanakan v...    1.0   
...                                                 ...    ...   
998   grab luncurkan pusat vaksinasi ramah penyandan...    1.0   
999   cegah penyebaran covid sentra vaksinasi kota s...    1.0   
1000  seneng liat animo teman teman makin banyak yan...    1.0   
1001  lonjakan kasus covid terjadi kemenkes hanya iz...    0.0   
1002  asa pernah dibahas awal ya militer diperbantuk...    1.0   

                                           tweet_tokens  \
0       [polres, tanjung, balai, laksanakan, vaksinasi]   
1     [dukung, program, presiden, jokowi, juta, vaks...   
2     [panglima, tni, marsekal, tni, hadi, tjahjanto...   
3     [polda, lampung, menggelar, vaksinasi, covid, ...   
4     [bupati, kobar, apresiasi, perusahaan, laksana...   
...                                                 ...   
998   [grab, luncurkan, pusat, vaksinasi, ramah, pen...   
999   [cegah, penyebaran, covid, sentra, vaksinasi, ...   
1000  [seneng, liat, animo, teman, teman, makin, ban...   
1001  [lonjakan, kasus, covid, terjadi, kemenkes, ha...   
1002  [asa, pernah, dibahas, awal, ya, militer, dipe...   

                                       tweet_tokens_WSW  
0               [tanjung, balai, laksanakan, vaksinasi]  
1     [dukung, program, juta, vaksinasi, lumajang, g...  
2               [menyebut, lonjakan, covid, terkendali]  
3     [lampung, menggelar, vaksinasi, covid, anggota...  
4     [bupati, kobar, apresiasi, perusahaan, laksana...  
...                                                 ...  
998   [grab, luncurkan, pusat, vaksinasi, ramah, pen...  
999   [cegah, penyebaran, covid, sentra, vaksinasi, ...  
1000  [seneng, liat, animo, teman, teman, divaksin, ...  
1001  [lonjakan, covid, izinkan, vaksin, vaksinasi, ...  
1002  [dibahas, militer, diperbantukan, vaksinasi, t...  

[792 rows x 4 columns]

In [15]:
# import Sastrawi package
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter


# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in df['tweet_tokens_WSW']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
            
print(len(term_dict))
print("------------------------")

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,":" ,term_dict[term])
    
print(term_dict)
print("------------------------")


# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

df['tweet_tokens_stemmed'] = df['tweet_tokens_WSW'].swifter.apply(get_stemmed_term)
print(df['tweet_tokens_stemmed'])

2691
------------------------
tanjung : tanjung
balai : balai
laksanakan : laksana
vaksinasi : vaksinasi
dukung : dukung
program : program
juta : juta
lumajang : lumajang
gelar : gelar
covid : covid
massal : massal
menyebut : sebut
lonjakan : lonjak
terkendali : kendali
lampung : lampung
menggelar : gelar
anggota : anggota
keluarga : keluarga
rumah : rumah
sakit : sakit
jalan : jalan
pramuka : pramuka
rajabasa : rajabasa
kota : kota
bandar : bandar
bupati : bupati
kobar : kobar
apresiasi : apresiasi
perusahaan : usaha
gotong : gotong
royong : royong
karyawan : karyawan
columbia : columbia
asia : asia
indonesia : indonesia
mendukung : dukung
pengaman : kam
pelaksanaan : laksana
warga : warga
usia : usia
makamhaji : makamhaji
bertempat : tempat
brigadir : brigadir
ruri : ruri
setiawan : setiawan
kartasura : kartasura
koramil : koramil
melaksanakan : laksana
pengamanan : aman
kegiatan : giat
kalbar : kalbar
serius : serius
lansia : lansia
salah : salah
banget : banget
ujian : uji
menangis

yaya : yaya
paham : paham
ketersediaan : sedia
tetep : tetep
panik : panik
lonjakannya : lonjak
god : god
sake : sake
jalanan : jalan
liat : liat
kuli : kuli
gapake : gapake
all : all
desperate : desperate
wajibkan : wajib
ampun : ampun
masy : masy
mensosialisasikan : sosialisasi
digalakan : gala
olh : olh
stakeholder : stakeholder
terkait : kait
memaksimalkan : maksimal
mengurangi : kurang
menekan : tekan
laju : laju
wapres : wapres
pemkot : pemkot
tangerang : tangerang
menimbulkan : timbul
kerumunan : kerumun
ombudsman : ombudsman
kepri : kepri
taknis : taknis
dikaji : kaji
ulang : ulang
melaporkan : lapor
dokter : dokter
medis : medis
tertangkap : tangkap
sinovac : sinovac
doctors : doctors
med : med
staff : staff
get : get
despite : despite
biddokkes : biddokkes
tubaba : tubaba
tekan : tekan
tau : tau
info : info
makasi : makas
nasional : nasional
kebanyakan : banyak
salahnya : salah
wni : wni
_ : 
manado : manado
pulang : pulang
dilurusin : dilurusin
dikit : dikit
acara : acara
be

keseeel : keseeel
napsu : napsu
makan : makan
hilang : hilang
mulutpun : mulut
menolak : tolak
mengunyah : kunyah
kapolres : kapolres
metro : metro
selatan : selatan
menghadiri : hadir
rapat : rapat
dikantor : kantor
walikota : walikota
gampang : gampang
web : web
seputar : putar
liatnya : liat
satuan : satu
brimob : brimob
pidie : pidie
tracking : tracking
zona : zona
merah : merah
pengusaha : usaha
programkan : program
sehari : hari
grab : grab
hadirkan : hadir
ramah : ramah
penyandang : sandang
disabilitas : disabilitas
yogyakarta : yogyakarta
muda : muda
karna : karna
risikonya : risiko
tua : tua
ngeri : ngeri
ngebludak : ngebludak
spot : spot
dluan : dluan
alasan : alas
tunggu : tunggu
adaaaa : adaaaa
milih : milih
jasa : jasa
keuangan : uang
digelar : gelar
tennis : tennis
indoor : indoor
senayan : senayan
tts : tts
kondisi : kondisi
langka : langka
mengakibatkan : akibat
trombositopenia : trombositopenia
lampu : lampu
perburukan : buru
kematian : mati
hinga : hinga
menghindari :

sukabumj : sukabumj
bansos : bansos
blt : blt
dd : dd
boyolali : boyolali
randusari : randusari
teras : teras
solusinya : solusi
nganggur : nganggur
beternak : ternak
sapi : sapi
kambing : kambing
unggas : unggas
rapp : rapp
kuansing : kuansing
datangkan : datang
sinopharm : sinopharm
ws : ws
mahasiswi : mahasiswi
stay : stay
nanya : nanya
beraktivitas : aktivitas
sob : sob
engga : engga
kerabat : kerabat
mengalami : alami
kejadian : jadi
ikutan : ikut
kipi : kipi
tenang : tenang
sobatkom : sobatkom
sensasi : sensasi
buru : buru
buruan : buru
hoki : hoki
hokian : hoki
serasa : serasa
heboh : heboh
hebohan : heboh
ngisi : ngisi
krs : krs
akang : akang
ad : ad
secapa : capa
hegarmanah : hegarmanah
dipimpin : pimpin
wakapolres : wakapolres
tribrata : tribrata
dibawah : bawah
taun : taun
muspika : muspika
kemalang : malang
pengecekan : kece
gel : gel
kantor : kantor
tlogowatu : tlogowatu
klaten : klaten
only : only
way : way
back : back
takde : takde
patroli : patroli
sambung : sambung
pem

sualang : sualang
hebat : hebat
mudik : mudik
mutusin : mutusin
ndak : ndak
sertu : sertu
sutardi : sutardi
genengduwur : genengduwur
irjen : irjen
asep : asep
suhendar : suhendar
royal : royal
ambarukmo : ambarukmo
cikarageman : cikarageman
aipda : aipda
ugi : ugi
handoko : handoko
setu : setu
never : never
idea : idea
supriyanta : supriyanta
matroni : matroni
kp : kp
bulak : bulak
bunut : bunut
karangsambung : karangsambung
dongkrak : dongkrak
gairah : gairah
nu : nu
mpd : mpd
relawan : rawan
pembayarannya : bayar
mrupakn : mrupakn
kemekes : kemekes
prcepatan : prcepatan
memerhatikan : perhati
diperjualbelikan : diperjualbelikan
mengatur : atur
pengobatan : obat
indikasi : indikasi
mengizinkan : izin
merek : merek
skema : skema
hibah : hibah
ditetapkan : tetap
gr : gr
moderna : moderna
cansino : cansino
pfizer : pfizer
novavax : novavax
mantau : mantau
face : face
shield : shield
sok : sok
ko : ko
nyontohinnya : nyontohinnya
kasar : kasar
diterjunkan : terjun
pelatihan : latih
vaksin

menghancurkan : hancur
penampakan : tampak
detik : detik
prosesi : prosesi
jakartakeras : jakartakeras
babelan : babelan
puskes : puskes
rincian : rincian
gajadi : gajadi
jaktim : jaktim
kesel : kesel
jakut : jakut
keluhan : keluh
tangan : tangan
kaki : kaki
amin : amin
curug : curug
binong : bong
persiapan : siap
cilacap : cilacap
kebut : kebut
laporan : lapor
sumpah : sumpah
ngerti : ngerti
random : random
dkk : dkk
ban : ban
warganegaranya : warganegaranya
nggatau : nggatau
macem : macem
chill : chill
amrik : amrik
santuy : santuy
eropa : eropa
spanyol : spanyol
mencabut : cabut
pemakaian : pakai
cempedak : cempedak
muslih : muslih
team : team
jatinegara : jatinegara
pimpin : pimpin
akp : akp
christiandini : christiandini
hardianti : hardianti
digerakkan : gerak
ngaruh : ngaruh
kematiannya : mati
volunteer : volunteer
sayangnya : sayang
tawarkan : tawar
hadiah : hadiah
pengeras : keras
suara : suara
rantau : rantau
pandan : pandan
gusdianto : gusdianto
mengajak : ajak
halal : halal


Pandas Apply:   0%|          | 0/792 [00:00<?, ?it/s]

0                    [tanjung, balai, laksana, vaksinasi]
1       [dukung, program, juta, vaksinasi, lumajang, g...
2                         [sebut, lonjak, covid, kendali]
3       [lampung, gelar, vaksinasi, covid, anggota, ke...
4       [bupati, kobar, apresiasi, usaha, laksana, vak...
                              ...                        
998     [grab, luncur, pusat, vaksinasi, ramah, sandan...
999     [cegah, sebar, covid, sentra, vaksinasi, kota,...
1000    [neng, liat, animo, teman, teman, vaksin, info...
1001    [lonjak, covid, izin, vaksin, vaksinasi, mandiri]
1002    [bahas, militer, bantu, vaksinasi, tenaga, med...
Name: tweet_tokens_stemmed, Length: 792, dtype: object


<ipython-input-15-4a5ae89cc3e7>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tweet_tokens_stemmed'] = df['tweet_tokens_WSW'].swifter.apply(get_stemmed_term)


In [ ]:
df

In [16]:
df = df.to_csv(r'C:\WPy64-3880\notebooks\AnalisisSentimen_VaksinasiSVM\datasetbismillah\tweetclean.csv', index = None, header=True) 

## Split Data

In [13]:
df = pd.read_csv("dataready-lex.csv")

In [14]:
df

tweets  sentiment
0         ['tanjung', 'balai', 'laksana', 'vaksinasi']        0.0
1    ['dukung', 'program', 'juta', 'vaksinasi', 'lu...        1.0
2              ['sebut', 'lonjak', 'covid', 'kendali']        1.0
3    ['lampung', 'gelar', 'vaksinasi', 'covid', 'an...        0.0
4    ['bupati', 'kobar', 'apresiasi', 'usaha', 'lak...        1.0
..                                                 ...        ...
787  ['grab', 'luncur', 'pusat', 'vaksinasi', 'rama...        1.0
788  ['cegah', 'sebar', 'covid', 'sentra', 'vaksina...        1.0
789  ['neng', 'liat', 'animo', 'teman', 'teman', 'v...        1.0
790  ['lonjak', 'covid', 'izin', 'vaksin', 'vaksina...        0.0
791  ['bahas', 'militer', 'bantu', 'vaksinasi', 'te...        1.0

[792 rows x 2 columns]

In [15]:
from sklearn import model_selection
from sklearn.model_selection import train_test_split

In [16]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(df['tweets'], df['sentiment'], test_size=0.1, random_state=0)

train_df = pd.DataFrame()
train_df['tweets'] = x_train
train_df['sentiment'] = y_train
test_df = pd.DataFrame()
test_df['tweets'] = x_test
test_df['sentiment'] = y_test

In [6]:
train_df

tweets  sentiment
644  ['personil', 'tebing', 'koramil', 'puskesmas',...        1.0
350  ['alam', 'vaksinasi', 'dekat', 'cemput', 'gamp...        1.0
733  ['vaksinasi', 'massal', 'stadion', 'cibinong',...        1.0
653  ['vaksin', 'covid', 'gratis', 'lokasi', 'vaksi...        1.0
504  ['pasongsongan', 'laku', 'cepat', 'vaksinasi',...        1.0
..                                                 ...        ...
763  ['sagita', 'putar', 'giat', 'ambu', 'bupati', ...        1.0
192                       ['syarat', 'bawa', 'lansia']        1.0
629  ['menkes', 'pasti', 'tambah', 'ribu', 'dosis',...        1.0
559  ['vaksinasi', 'dikebutttttt', 'juta', 'hari', ...        1.0
684  ['vaksinasi', 'gratis', 'panti', 'waluyo', 'sa...        1.0

[712 rows x 2 columns]

In [7]:
test_df

tweets  sentiment
625  ['tinjau', 'vaksinasi', 'massal', 'stadion', '...        1.0
142  ['kon', 'tau', 'tha', 'kemudi', 'bluebird', 's...        0.0
700  ['moga', 'mudah', 'vaksinasi', 'mama', 'nnt', ...        1.0
55   ['vaksin', 'cek', 'rapid', 'antibodi', 'hasil'...        1.0
499  ['pemkot', 'jakpus', 'kolaborasi', 'bareng', '...        1.0
..                                                 ...        ...
669  ['canang', 'ajar', 'tatap', 'muka', 'ptm', 'ba...        1.0
375  ['panik', 'gak', 'engga', 'kerabat', 'salah', ...        0.0
497  ['udh', 'colong', 'kontak', 'positif', 'pcr', ...        1.0
717  ['tinjau', 'vaksinasi', 'massal', 'stadion', '...        1.0
253  ['bang', 'rencana', 'ntb', 'vaksinasi', 'masal...        1.0

[80 rows x 2 columns]

## TF-IDF

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(use_idf=True)
corpus = df['tweets']
x = tfidf_vect.fit(corpus)
x_train_tfidf = x.transform(x_train)
x_test_tfidf = x.transform(x_test)

In [ ]:
# print(x_test_tfidf)

# tfidf_vect.get_feature_names()

# x_train_tfidf.todense()

# df = pd.DataFrame(x_train_tfidf.todense().T,
#                  index=tfidf_vect.get_feature_names(),
#                  columns=[f'D{i+1}' for i in range(len(x_train))])
# df

# df = df.to_csv(r'C:\WPy64-3880\notebooks\AnalisisSentimen_VaksinasiSVM\datasetbismillah\tfidf-X_train.csv', header=True) 

## Perform SVM

In [22]:
from sklearn import svm

SVM = svm.SVC(kernel='linear')
SVM.fit(x_train_tfidf, y_train)

SVC(kernel='linear')

## Eval Model

In [23]:
SVM.score(x_test_tfidf, y_test)

0.7875

In [24]:
predictions_SVM = SVM.predict(x_test_tfidf)
# print(predictions_SVM)

In [25]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

confusion_matrix = confusion_matrix(y_test, predictions_SVM)

print('Confusion Matrix : ') 
print(confusion_matrix)
print('\n Report Hasil : ') 
print(classification_report(y_test, predictions_SVM))

Confusion Matrix : 
[[ 5 17]
 [ 0 58]]

 Report Hasil : 
              precision    recall  f1-score   support

         0.0       1.00      0.23      0.37        22
         1.0       0.77      1.00      0.87        58

    accuracy                           0.79        80
   macro avg       0.89      0.61      0.62        80
weighted avg       0.84      0.79      0.73        80



## Visualisasi

In [10]:
import seaborn as sns
import matplotlib.pyplot as plt

f, ax = plt.subplots(figsize=(8,5))
sns.heatmap(confusion_matrix, annot=True, fmt=".0f", ax=ax)
plt.xlabel("prediksi")
plt.ylabel("aktual")
plt.show()

In [11]:
import matplotlib.pyplot as plt

values = [58, 22]
explode = [0, 0.1]
labels = ['Positif', 'Negatif']
colors = ['blue', 'red']

def make_autopct(values):
    def my_autopct(pct):
        total  = sum(values)
        val = int(round(pct*total/100.0))
        return '{p:.2f}%({v:d})'.format(p=pct,v=val)
    return my_autopct

plt.pie(values, colors=colors, labels=labels, explode=explode,
      autopct=make_autopct(values), shadow=True, startangle=90)
plt.legend()
plt.title("Hasil Prediksi Sentimen Vaksinasi COVID-19")
plt.show()